# BenSAF Workflow Example
This notebook demonstrates the use of BenSAF to execute a health benefits analysis on the use of alternative aviation fuels. The notebook reproduces a simplified version of the ORD analysis.

## 1. Initialization
Import modules and set options

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np

import bensaf

In [ ]:
USE_SYNTHETIC_DATA = False

Create output directory and load tracts

In [ ]:
output_dir = Path("output", "saf_workflow_example")
output_dir.mkdir(parents=True, exist_ok=True)

Load in tract data, exposure data, and mortality data. If `USE_SYNTHETIC_DATA` is true random  values for the necessary columns are generated, otherwise real data is loaded (assuming it is available).

In [ ]:
if USE_SYNTHETIC_DATA:
    tracts_file = Path("..", "sources", "melissa_ord", "data", "Illinois Shapefile Bounded")
    tracts = gpd.read_file(tracts_file)
    tracts_gdf, exposure_df, mortality_df = bensaf.utils.create_synthetic_data(
        tracts_gdf=tracts
    )
else: # Load from processed data
    tracts_gdf = gpd.read_file('../data/converted/tracts_gdf.geojson')
    exposure_df = pd.read_csv('../data/converted/exposure_df.csv')
    mortality_df = pd.read_csv('../data/converted/mortality_df.csv')

Preview tract data

In [ ]:
tracts_gdf.head(3)

In [ ]:
tracts_gdf.plot()

Preview exposure data

In [ ]:
exposure_df.head(3)

Preview mortality data

In [ ]:
mortality_df.head(3)

Define the workflow configuration. This configuration file contains various settings and parameters used by the workflow class to execute the analysis. The config can alternatively be loaded from a JSON file.

In [ ]:
config = {
    'control_scenarios': [5, 25, 50],
    'demographic_columns': ['poc_proportion', 'low_income_proportion'],
    'airport_coordinates': (-87.90472, 41.97861)
}


Initialize the workflow class using the config file

In [ ]:
workflow = bensaf.workflow.Workflow(config)

## 2. Process Data

Add the data to the workflow

In [ ]:
workflow.load_tract_data(tracts_gdf)
workflow.load_exposure_data(exposure_df)
workflow.load_mortality_data(mortality_df)

Use the `prepare_data` method to combine the mortality and exposure data with the tract data to create a unified GeoDataFrame accessed at `workflow.analysis_data`.

Additionally, some initial data preprocessing is performed such as computing natural mortality rate per captia from the natural mortality rate.

In [ ]:
workflow.prepare_data()
workflow.analysis_data.head(3)

Add discrete distance categories to the tracts using the `bin_tracts_by_distance` method.

In [ ]:
workflow.bin_tracts_by_distance([0, 10, 20, 100])

In [ ]:
workflow.analysis_data.plot("distance_bin")

Aggregated statistics can be easily accessed using Pandas' `groupby` method. Total population within each distance bin category is shown below.

In [ ]:
workflow.analysis_data.groupby("distance_bin")["population"].sum()

Plot boxplots of the tract populations within each distance bin. 

In [ ]:
pop10km =workflow.analysis_data.loc[workflow.analysis_data["distance_bin"] == "0-10 km", "population"]
pop1020km =workflow.analysis_data.loc[workflow.analysis_data["distance_bin"] == "10-20 km", "population"]
pop20km =workflow.analysis_data.loc[workflow.analysis_data["distance_bin"] == "20+ km", "population"]

bp = plt.boxplot([pop10km, pop1020km, pop20km], positions = [1, 2, 3], widths=0.95, showfliers=False, patch_artist=True);

plt.xlim([-0.1, 4.1])
plt.xticks([1, 2, 3], ['<10 km', '10-20 km', '>20 km'])
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.title('Tract Population by Distance Bin', fontsize=15)


The `create_choropleth_map` function can be used to plot variables geospatially across tracts.

In [ ]:
from bensaf.graphics import create_choropleth_map

create_choropleth_map(
    workflow.analysis_data, 
    "population", 
    "Total Population",
    point_locations=workflow.config['airport_coordinates'],
    vmin=0,
    vmax=10000,
    cmap='Blues',
    show_background=True,
    show_boundaries=True,
)

In [ ]:
create_choropleth_map(
    workflow.analysis_data, 
    "low_income_proportion", 
    "Low Income Proportion",
    point_locations=workflow.config['airport_coordinates'],
    vmin=0,
    vmax=1,
    cmap='Reds',
    show_background=True,
    show_boundaries=True,
)

## 3. Execute Analysis

### Apply control scenarios

The `apply_control_scenarios` method uses the scenario percentages defined in config to generate UFP reduction scenarios. This produces a dictionary of scenarios `workflow.control_scenarios`. The "data" key of each scenario dictionary contains a dataframe with the reduced and delta concentrations for each tract. Later in the analysis, other scenario relevant values are added to this dictionary.

In [ ]:
workflow.apply_control_scenarios()
workflow.control_scenarios[5] # 5 reduction scenario

Plot pollutant concentration maps for the baseline and three reduction cases.

In [ ]:
# Create 2x2 grid of maps using choropleth_map
fig, axs = plt.subplots(2, 2, figsize=(15, 12))

# Define the variables to map
control_scenarios = workflow.control_scenarios
variables = [
    (workflow.analysis_data["pollutant_concentration"], "Baseline"),
    (control_scenarios[5]["data"]["reduced_concentration_5"], "5% Reduction"),
    (control_scenarios[25]["data"]["reduced_concentration_25"], "25% Reduction"),
    (control_scenarios[50]["data"]["reduced_concentration_50"], "50% Reduction"),
]

# Calculate global min and max for consistent color scaling
all_values = []
for column_data, _ in variables:
    all_values.extend(column_data.values)

global_vmin = min(all_values)
global_vmax = max(all_values)

print(f"Global color range: {global_vmin:.2f} to {global_vmax:.2f}")

# Create each map
for i, (column_data, title) in enumerate(variables):
    row = i // 2
    col = i % 2
    ax = axs[row, col]
    
    # Use create_choropleth_map function
    bensaf.graphics.create_choropleth_map(
        gdf=workflow.analysis_data,
        column=column_data,
        title=title,
        vmin=global_vmin,
        vmax=global_vmax,
        point_locations=workflow.config['airport_coordinates'],
        ax=ax
    )
    
plt.tight_layout()
plt.show()

### Calculate health impacts

The `calculate_health_impacts` method uses the default health impact function from Bouma et al to compute:
- Relative risk
- Attributable fraction
- Attributable cases
- Attributable mortality

for each scenario.

In [ ]:
workflow.calculate_health_impacts()

After running the method, the `control_scenarios` dictionaries  will be populated with health impact data. 

In [ ]:
workflow.control_scenarios[5].keys()

In [ ]:
workflow.control_scenarios[25]['attributable_cases_25']

## 4. Explore results

Print study population characteristics.

In [ ]:
print('Study Population: '+str(round(np.sum(workflow.analysis_data['population']))))

print('Low Income Population: '+str(round(np.sum(workflow.analysis_data['low_income_population']))))
print('Low Income Population Proportion: '+str(round(100*np.sum(workflow.analysis_data['low_income_population'])/np.sum(workflow.analysis_data['population']), 2)))

print('POC Population: '+str(round(np.sum(workflow.analysis_data['poc_population']))))
print('POC Population Proportion: '+str(round(100*np.sum(workflow.analysis_data['poc_population'])/np.sum(workflow.analysis_data['population']), 2)))

Print weighted UFP values.

In [ ]:
print('Average UFP: '+str(round(workflow.analysis_data['pollutant_concentration'].mean(), 1))+' pt/cm\u00b3')

print('Low Income Population-Weighted UFP: '+str(round(bensaf.utils.calculate_weighted_ufp(workflow.analysis_data['pollutant_concentration'], workflow.analysis_data['low_income_population']), 1))+' pt/cm\u00b3')
print('Not Low Income Population-Weighted UFP: '+str(round(bensaf.utils.calculate_weighted_ufp(workflow.analysis_data['pollutant_concentration'], workflow.analysis_data['not_low_income_population']), 1))+ ' pt/cm\u00b3')

print('POC Population-Weighted UFP: '+str(round(bensaf.utils.calculate_weighted_ufp(workflow.analysis_data['pollutant_concentration'], workflow.analysis_data['poc_population']), 1))+' pt/cm\u00b3')
print('Non POC Population-Weighted UFP: '+str(round(bensaf.utils.calculate_weighted_ufp(workflow.analysis_data['pollutant_concentration'], workflow.analysis_data['population']-workflow.analysis_data['poc_population']), 1))+ ' pt/cm\u00b3')

Print Attributable case reductions.

In [ ]:
print('Attributable Case Reductions for Entire Population (5% SAF Blend): '+str(round(workflow.control_scenarios[5]['attributable_cases_5']))+' ('+str(round(workflow.control_scenarios[5]['lower_attributable_cases_5']))+', '+str(round(workflow.control_scenarios[5]['upper_attributable_cases_5']))+')')
print('Attributable Case Reductions for Entire Population (25% SAF Blend): '+str(round(workflow.control_scenarios[25]['attributable_cases_25']))+' ('+str(round(workflow.control_scenarios[25]['lower_attributable_cases_25']))+', '+str(round(workflow.control_scenarios[25]['upper_attributable_cases_25']))+')')
print('Attributable Case Reductions for Entire Population (50% SAF Blend): '+str(round(workflow.control_scenarios[50]['attributable_cases_50']))+' ('+str(round(workflow.control_scenarios[50]['lower_attributable_cases_50']))+', '+str(round(workflow.control_scenarios[50]['upper_attributable_cases_50']))+')')